In [ ]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import os

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("----------------------------------------------------------------")
print("  Case Study: Dense vs RNN vs LSTM for Time Series Forecasting  ")
print("----------------------------------------------------------------")

# ==========================================
# 1. Data Acquisition (Open-Meteo API)
# ==========================================
def fetch_weather_data(days=90):
    """
    Fetches hourly temperature data for Tokyo (Latitude: 35.6895, Longitude: 139.6917)
    for the past 'days' days.
    """
    print(f"\n[1/5] Fetching last {days} days of weather data for Tokyo...")
    
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=days)
    
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": 35.6895,
        "longitude": 139.6917,
        "start_date": start_date.strftime("%Y-%m-%d"),
        "end_date": end_date.strftime("%Y-%m-%d"),
        "hourly": "temperature_2m",
        "timezone": "Asia/Tokyo"
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Parse into DataFrame
        df = pd.DataFrame({
            'time': pd.to_datetime(data['hourly']['time']),
            'temp': data['hourly']['temperature_2m']
        })
        
        print(f"      Successfully fetched {len(df)} data points.")
        return df
    except Exception as e:
        print(f"      Error fetching data: {e}")
        # Fallback to synthetic sine wave data if API fails
        print("      Falling back to synthetic data...")
        t = np.linspace(0, 100, 2000)
        y = np.sin(t) + np.random.normal(0, 0.1, 2000)
        return pd.DataFrame({'temp': y})

# ==========================================
# 2. Data Preprocessing
# ==========================================
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Fetch Data
df = fetch_weather_data()
values = df['temp'].values.reshape(-1, 1)

# Normalize data (Neural Networks converge faster with 0-1 scaling)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values)

# Create sequences: Use past 24 hours to predict next hour
SEQ_LENGTH = 24
X, y = create_sequences(scaled_values, SEQ_LENGTH)

# Split into Train (80%) and Test (20%)
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"\n[2/5] Data Prepared:")
print(f"      Training samples: {X_train.shape[0]}")
print(f"      Testing samples:  {X_test.shape[0]}")
print(f"      Input shape:      {X_train.shape} (Samples, Timesteps, Features)")

# ==========================================
# 3. Model Definitions
# ==========================================

def build_dense_model(input_shape):
    """
    Standard Feed-Forward Network. 
    It flattens the time dimension, effectively ignoring the sequence order logic.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_rnn_model(input_shape):
    """
    Simple RNN. 
    Processes sequence step-by-step but often struggles with vanishing gradients.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.SimpleRNN(50, activation='tanh', input_shape=input_shape),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_lstm_model(input_shape):
    """
    LSTM (Long Short-Term Memory).
    Has gates (input, output, forget) to manage long-term dependencies.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(50, activation='tanh', input_shape=input_shape),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# ==========================================
# 4. Training & Comparison
# ==========================================
BATCH_SIZE = 32
EPOCHS = 20

input_shape = (SEQ_LENGTH, 1)

results = {}
models = {
    "Dense (NN)": build_dense_model(input_shape),
    "Simple RNN": build_rnn_model(input_shape),
    "LSTM":       build_lstm_model(input_shape)
}

print(f"\n[3/5] Training Models ({EPOCHS} epochs each)...")

for name, model in models.items():
    print(f"      Training {name}...", end=" ")
    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1,
        verbose=0  # Silent training
    )
    
    # Predict
    predicted = model.predict(X_test, verbose=0)
    
    # Inverse transform to get actual temperature values
    predicted_actual = scaler.inverse_transform(predicted)
    y_test_actual = scaler.inverse_transform(y_test)
    
    # Calculate Error
    mse = mean_squared_error(y_test_actual, predicted_actual)
    results[name] = {
        "mse": mse,
        "preds": predicted_actual,
        "history": history.history['loss']
    }
    print(f"Done. MSE: {mse:.4f}")

# ==========================================
# 5. Visualization & Analysis
# ==========================================
print("\n[4/5] Generating Plots...")

# Plot 1: Prediction Comparison (Zoomed in on 100 hours)
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
subset_n = 100 # Look at first 100 hours of test set for clarity
plt.plot(scaler.inverse_transform(y_test)[:subset_n], label='Actual Data', color='black', linewidth=2, linestyle='--')

colors = {'Dense (NN)': 'red', 'Simple RNN': 'orange', 'LSTM': 'green'}
styles = {'Dense (NN)': ':', 'Simple RNN': '-.', 'LSTM': '-'}

for name, data in results.items():
    plt.plot(data['preds'][:subset_n], label=f'{name} (MSE: {data["mse"]:.2f})', color=colors[name], linestyle=styles[name])

plt.title(f'Model Comparison: Predicting Tokyo Temperature (Next Hour)\nLower MSE is better', fontsize=14)
plt.ylabel('Temperature (°C)')
plt.xlabel('Time Steps (Hours)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Error Bar Chart
plt.subplot(2, 1, 2)
names = list(results.keys())
errors = [results[n]['mse'] for n in names]
bars = plt.bar(names, errors, color=['red', 'orange', 'green'])

plt.title('Mean Squared Error Comparison (Lower is Better)')
plt.ylabel('MSE')

# Add text on bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 4), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('model_comparison_results.png')
print(f"      Plot saved to 'model_comparison_results.png'")

print("\n[5/5] Final Analysis:")
sorted_results = sorted(results.items(), key=lambda x: x[1]['mse'], reverse=True)
for i, (name, data) in enumerate(sorted_results):
    rank = i + 1
    print(f"      Rank {rank}: {name} | Error: {data['mse']:.4f}")

print("\nInterpretation:")
print("1. Dense NN: Treats input as a flat vector. Ignores the 'sequence' nature of time.")
print("2. RNN: Remembers state, but struggles with vanishing gradients over time.")
print("3. LSTM: Advanced gating mechanisms allow it to retain relevant long-term patterns best.")
print("----------------------------------------------------------------")